In [ ]:
import os
import json
import pyautogui
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

In [ ]:
from src.chrome_manager import select_mobile_and_get_driver
from src.account_manager import login, save_cookies
from src.utils import x_path_click
from resources.xpath_dict import xpath_dict, id_dict
from src.naverband_automation import roof_bands, perform_logout
from config import *

In [ ]:
import pyautogui    # prompt 입력에 필요
import time         # sleep, 시간 계산 등에 필요
import random       # rand_sleep, 셔플 등에 필요
from selenium import webdriver   # driver 사용에 필요
from selenium.webdriver.common.alert import Alert   # Alert 처리 필요
from selenium.webdriver.common.keys import Keys     # 키입력(ESC, ENTER)에 필요

from src.chrome_manager import select_mobile_and_get_driver
from src.account_manager import login, save_cookies
from src.utils import x_path_click
from resources.xpath_dict import xpath_dict, id_dict
from src.naverband_automation import roof_bands, perform_logout
from config import *

### mobile, driver = select_mobile_and_get_driver() 부분


In [ ]:
# 1) id_dict에서 전화번호 키 리스트 추출
ids = list(id_dict.keys())
print(ids)

In [ ]:
from src.utils import resource_path, ensure_dir, get_cookie_path, x_path_click, x_path_send_keys, logger, move_mouse_naturally
# 2) pyautogui로 전화번호 선택
mobile_num = pyautogui.confirm('전화번호를 선택하시오', buttons=ids)
logger.info(f'선택한 전화번호는 {mobile_num}')

In [ ]:
from src.utils import get_profile_path
# 3) 선택된 전화번호에 맞는 크롬 프로필 경로 계산 및 생성
profile_dir = get_profile_path(mobile_num)

In [ ]:
from src.chrome_manager import fix_chrome_profile_preferences
 # 4) 프로필 Preferences 파일 패치
fix_chrome_profile_preferences(profile_dir)

In [ ]:
# 5) ChromeOptions 객체 생성 및 옵션 추가
options = webdriver.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_dir}")  # 프로필 적용
logger.info(f'profile적용')

In [ ]:
# 로그레벨 설정 (Print 로그 최소화)
options.add_argument('--log-level=3')
logger.info(f'print-log-최소화')

In [ ]:
# GPU 비활성화 옵션
options.add_argument('--disable-gpu')
options.add_argument('--disable-software-rasterizer')
logger.info(f'GPU비활성화')

In [ ]:
# 탐지 회피 관련 옵션 추가
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
logger.info('탐지관련옵션추가')

In [ ]:
for opt in DEFAULT_CHROME_OPTIONS:
    options.add_argument(opt)
    print(opt) # 기본 옵션
    
    
                               
options.add_argument(f"user-agent={USER_AGENT}")       # 사용자 에이전트

In [ ]:

# 5-1)======비밀번호 저장/자동완성 관리 팝업 사전 차단 ======
# 이 프리퍼런스(prefs) 옵션을 추가하면 로그인 후 비밀번호 저장 안내창이 더 이상 나타나지 않음
prefs = {
    "credentials_enable_service": False,            # 크롬 비밀번호 관리 서비스 OFF
    "profile.password_manager_enabled": False       # 프로필 비밀번호 저장/자동완성 OFF
}
options.add_experimental_option("prefs", prefs)

# 6) ChromeDriverManager로 드라이버 설치/업데이트 후 실행
service = Service(ChromeDriverManager().install(), log_path='NUL')  # 윈도우 기준 콘솔 로그 숨김
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# 7) 네이버밴드 로그인 페이지로 자동 이동
driver.get(NAVERBAND_URL)

In [ ]:
# 탐지 JS 속성 우회 강화 (execute_cdp_cmd 사용)
try:
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {get: () => false});
            Object.defineProperty(navigator, 'plugins', {
                get: () => [1, 2, 3, 4, 5],
            });
            Object.defineProperty(navigator, 'languages', {
                get: () => ['ko-KR', 'ko'],
            });
        """
    })
except Exception as e:
        logger.error(f"[WARN] 탐지 JS 우회 코드 주입 실패: {e}")

In [ ]:
# 8) 드라이버에 선택된 전화번호 정보 저장
driver.selected_mobile = mobile_num

from src.chrome_manager import close_restore_popup
# 9) "페이지 복원" 팝업 자동 닫기 시도
close_restore_popup(driver)

In [ ]:
mobile, driver = select_mobile_and_get_driver()

### login(driver)부분

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
account_id = driver.selected_mobile
wait = WebDriverWait(driver, 10)

In [ ]:
from src.account_manager import load_cookies
# 1. 쿠키 인증 복원(도메인별 add_cookie)
load_cookies(driver)
time.sleep(1)

In [ ]:
# 2. 로그인 페이지로 이동
driver.get(NAVERBAND_LOGIN_URL)
time.sleep(2)

In [ ]:
from selenium.webdriver.common.by import By
from src.utils import x_path_send_keys
from src.utils import move_mouse_naturally
from src.utils import resource_path, ensure_dir, get_cookie_path, x_path_click, x_path_send_keys, logger, move_mouse_naturally
# 3. 전화번호 입력창(XPath) 실존 체크
log_in_xpath = xpath_dict['log_in']
if driver.find_elements(By.XPATH, log_in_xpath):
    x_path_send_keys(driver, log_in_xpath, account_id)
    move_mouse_naturally()
    x_path_send_keys(driver, log_in_xpath, Keys.RETURN)
    logger.info("[INFO] 전화번호 입력 성공")
else:
    logger.warning("[WARN] 입력창 없음(자동 인증/사이트 구조 변경 등)")
    # return

In [ ]:
from selenium.webdriver.common.by import By
from src.utils import x_path_send_keys
from src.utils import move_mouse_naturally
from src.utils import resource_path, ensure_dir, get_cookie_path, x_path_click, x_path_send_keys, logger, move_mouse_naturally
# 4. 비밀번호 입력도 동일하게 처리
password_xpath = xpath_dict['password']
if driver.find_elements(By.XPATH, password_xpath):
    x_path_send_keys(driver, password_xpath, id_dict[account_id])
    move_mouse_naturally()
    x_path_send_keys(driver, password_xpath, Keys.RETURN)
    logger.info("[INFO] 비밀번호 입력 성공")
else:
    logger.warning("[WARN] 비밀번호 입력창 없음!")
    # return

In [ ]:
login(driver)

In [ ]:
def process_band(driver, xpath_dict, i, TXT_DIR, IMAGE_DIR):
    """
    밴드 작업을 사람 행동처럼 자연스럽게 수행
    """
    # 브라우저 창 포커스 유지 (윈도우창 제목 예시)
    focus_window("band")
    i ='//*[@id="content"]/section/div[2]/div/ul/li[2]/div/div/a/div[1]/div/span'
    # 마우스 자연 이동 후 밴드 클릭
# 마우스 자연 이동 후 밴드 클릭
    move_mouse_naturally()
    x_path_click(driver,i)
    human_delay("click")

    # 글쓰기 버튼 클릭 + 고민 시간
    human_delay("thinking")
    safe_xpath_click(driver, xpath_dict['글쓰기_1'])

    # 텍스트 입력 (느린 타이핑 포함)
    write_text_from_folder(driver, xpath_dict['글쓰기_2'], TXT_DIR)
    human_delay("typing")

    # 이미지 업로드 + 대기
    upload_file_from_folder(driver, IMAGE_DIR)
    human_delay("upload")

    # 이미지 첨부 클릭 (마우스 이동 포함)
    move_mouse_naturally()
    x_path_click(driver, xpath_dict['이미지첨부'])
    human_delay("click")

    # 게시하기 클릭 + 생각시간
    x_path_click(driver, xpath_dict['이미지게시'])
    human_delay("thinking")

    # 화면 스크롤 후 홈으로 돌아가기
    driver.execute_script("window.scrollBy(0, window.innerHeight / 3)")
    human_delay("scroll")
    x_path_click(driver, xpath_dict['홈'])
    human_delay("scroll")

In [ ]:
process_band(driver, xpath_dict, i, TXT_DIR, IMAGE_DIR)

In [ ]:
import pickle

with open('accounts/01075381965/cookies/cookies.pkl', 'rb') as f:  # 경로와 파일명은 실제 존재하는 것으로 수정
    cookies = pickle.load(f)

for cookie in cookies:
    print(cookie)


In [ ]:
import pickle  # pickle 파일(바이너리)을 읽기 위한 모듈
import json    # json 파일(텍스트)을 저장하기 위한 모듈

# 1. pkl 파일 열기
with open('accounts/01075381965/cookies/cookies.pkl', 'rb') as f:  # 실제 파일 경로에 맞게 수정
    cookies = pickle.load(f)  # 리스트(dict)의 형태로 읽힘

# 2. json 파일로 저장
with open('accounts/01075381965/cookies/cookies.json', 'w', encoding='utf-8') as f:  # 저장 경로/이름 수정 가능
    json.dump(cookies, f, ensure_ascii=False, indent=4)  # 가독성 좋게 저장

print("pickle 파일이 json 파일로 변환되었습니다.")


### select_mobile_and_get_driver()

In [ ]:
from resources.xpath_dict import id_dict
import pyautogui
from src.utils import get_profile_path, logger, move_mouse_naturally
from src.chrome_manager import fix_chrome_profile_preferences, get_stealth_driver, close_restore_popup
from config import MOBILE_USER_AGENT_MAPPING, USER_AGENT, NAVERBAND_LOGIN_URL, NAVERBAND_URL
import time

In [ ]:
ids = list(id_dict.keys())
print(ids)

In [ ]:
mobile_num = pyautogui.confirm('전화번호를 선택하시오', buttons=ids)
logger.info(f'선택한 전화번호: {mobile_num}')

In [ ]:
profile_dir = get_profile_path(mobile_num)
fix_chrome_profile_preferences(profile_dir)

In [ ]:
user_agent = MOBILE_USER_AGENT_MAPPING.get(mobile_num, USER_AGENT)
logger.info(f"[INFO] 사용할 User-Agent: {user_agent}")

In [ ]:
driver = get_stealth_driver(profile_dir, user_agent)

In [ ]:
move_mouse_naturally()
driver.get(NAVERBAND_URL)
time.sleep(2)

In [ ]:
driver.selected_mobile = mobile_num
close_restore_popup(driver)

## 통합

In [ ]:
from src.chrome_manager import select_mobile_and_get_driver
from src.account_manager import login
from src.utils import logger


In [ ]:
mobile, driver = select_mobile_and_get_driver()

In [ ]:
login(driver)

In [ ]:
from src.utils import focus_window
focus_window("band")

In [ ]:
from src.utils import *
from src.naverband_automation import write_text_from_folder, upload_file_from_folder

In [ ]:
from src.utils import x_path_click
x_path_click(driver,'//*[@id="content"]/section/div[2]/div/ul/li[5]/div/div/a/div[1]/div/span' )

In [ ]:
human_delay("thinking")
x_path_human_click(driver, xpath_dict['글쓰기_1'])

In [ ]:
# 텍스트 입력 (느린 타이핑 포함)
write_text_from_folder(driver, xpath_dict['글쓰기_2'], TXT_DIR)
human_delay("typing")

In [ ]:
# 이미지 업로드 + 대기
upload_file_from_folder(driver, IMAGE_DIR)
human_delay("upload")

In [ ]:
# 이미지 첨부 클릭 (마우스 이동 포함)
move_mouse_naturally()
x_path_click(driver, xpath_dict['이미지첨부'])
human_delay("click")

In [ ]:
# 게시하기 클릭 + 생각시간
x_path_click(driver, xpath_dict['이미지게시'])
human_delay("thinking")

In [ ]:
# 화면 스크롤 후 홈으로 돌아가기
driver.execute_script("window.scrollBy(0, window.innerHeight / 3)")
human_delay("scroll")
x_path_click(driver, '//*[@id="header"]/div/div[1]/h1/a')
human_delay("scroll")


In [ ]:
import random, time

In [ ]:
n = 0
while n > 1 :
    roof_bands(
        driver, 
        xpath_dict, 
        BAND_LIST, 
        TXT_DIR, 
        IMAGE_DIR, 
        MAX_ERROR_CNT=3
    )
    rand_sleep = random.randrange(10,20)
    move_mouse_naturally()
    rel_time = time.strftime('%p %I시%M분', time.localtime(time.time()+rand_sleep))
    logger.info('-'*20)
    logger.info('{}회전, {}초간 휴식/ {}에 다시 시작합니다'.format(t, rand_sleep, rel_time))
    logger.info('-'*20) 
    time.sleep(rand_sleep)
    t += 1

In [ ]:
from src.naverband_automation import roof_bands
from config import *


In [ ]:
roof_bands(
    driver, 
    xpath_dict, 
    BAND_LIST, 
    TXT_DIR, 
    IMAGE_DIR, 
    MAX_ERROR_CNT=3
    )

In [ ]:
x_path_click(driver,'//*[@id="content"]/section/div[2]/div/ul/li[5]/div/div/a/div[1]/div/span')